# Introduction
Jack Wilson
10/26/2025

This notebook outlines the cleaning and merging of raw data into intermediate and final dataframes

# Import Modules

In [1]:
import pandas as pd

import os, sys, re

from datetime import timedelta

In [2]:
# Connects notebook to 'src' Package
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src.data_functions import load_id_map, save_id_map, constructor_mapping

# DataFrame Display Options

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)

# Prep for Merge

## ID Map

### Load

In [102]:
id_map = load_id_map('../data/raw/circuit_id_map.pkl')

### Fix Circuit ID Map
Austria & Styria (2020, 2021) are the same race

Great Britain & 70th Anniversary (2020) are the same race

In [103]:
# Ensure Austria and Styria have the same circuit ID
if 'Styria' in id_map and 'Austria' in id_map:
    id_map_styria = id_map['Styria']
    id_map['Styria'] = id_map['Austria']

# Ensure Great Britain and 70th Anniversary have the same circuit ID
if '70th Anniversary' in id_map and 'Great Britain' in id_map:
    id_map_anniversary = id_map['70th Anniversary']
    id_map['70th Anniversary'] = id_map['Great Britain']

print(f"Styria: {id_map_styria} -> {id_map.get('Styria')}")
print(f"Austria: {id_map.get('Austria')}")
print(f"70th Anniversary: {id_map_anniversary} -> {id_map.get('70th Anniversary')}")
print(f"Great Britain: {id_map.get('Great Britain')}")

Styria: 9 -> 9
Austria: 9
70th Anniversary: 10 -> 10
Great Britain: 10


### Save

In [104]:
save_id_map('../data/raw/circuit_id_map.pkl', id_map)

## Race Results 2001-2017

### Load

In [76]:
races_2001 = pd.read_csv('../data/raw/race_results_raw_2001-2017.csv')

### Convert Date

In [77]:
# Process date column
for i, date in enumerate(races_2001['date']):
    if '-' in date:
        races_2001.at[i, 'date'] = date.split('-')[1].strip()

# Convert to datetime
races_2001['date'] = pd.to_datetime(races_2001['date'], format='mixed')
races_2001.head(10)

,date,driver_id,position,driver_name,points
0,2001-03-04,1,1,Michael Schumacher,10.0
1,2001-03-04,2,2,David Coulthard,6.0
2,2001-03-04,3,3,Rubens Barrichello,4.0
3,2001-03-04,4,4,Nick Heidfeld,3.0
4,2001-03-04,5,5,Heinz-Harald Frentzen,2.0
5,2001-03-04,6,6,Kimi Räikkönen,1.0
6,2001-03-04,7,7,Olivier Panis,0.0
7,2001-03-04,8,8,Luciano Burti,0.0
8,2001-03-04,9,9,Jean Alesi,0.0
9,2001-03-04,10,10,Jos Verstappen,0.0


### Create Year and Round Columns
These columns will help after the merge with 2018+ data. Gives same features across both datasets to make sorting easier

In [78]:
# Create year column
races_2001['year'] = races_2001['date'].dt.year

# Create round column
races_2001 = races_2001.sort_values(['year', 'date'])
races_2001['round'] = races_2001.groupby('year')['date'].transform(lambda x: x.rank(method='dense').astype(int))

races_2001.head(25)

,date,driver_id,position,driver_name,points,year,round
0,2001-03-04,1,1,Michael Schumacher,10.0,2001,1
1,2001-03-04,2,2,David Coulthard,6.0,2001,1
2,2001-03-04,3,3,Rubens Barrichello,4.0,2001,1
3,2001-03-04,4,4,Nick Heidfeld,3.0,2001,1
4,2001-03-04,5,5,Heinz-Harald Frentzen,2.0,2001,1
5,2001-03-04,6,6,Kimi Räikkönen,1.0,2001,1
6,2001-03-04,7,7,Olivier Panis,0.0,2001,1
7,2001-03-04,8,8,Luciano Burti,0.0,2001,1
8,2001-03-04,9,9,Jean Alesi,0.0,2001,1
9,2001-03-04,10,10,Jos Verstappen,0.0,2001,1


### Convert Position to Numeric

In [79]:
# Converts position to numeric, or changes classification to numeric
def convert_position(row, prev_position=None):
    try:
        return int(row['position'])
    except (ValueError, TypeError):
        if prev_position is not None:
            return prev_position + 1
        else:
            return 1

# Apply conversion
prev_pos = None
converted_positions = []
for idx, row in races_2001.iterrows():
    current_pos = convert_position(row, prev_pos)
    converted_positions.append(current_pos)
    prev_pos = current_pos

races_2001['position'] = converted_positions
races_2001['position'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int64)

### Save

In [80]:
races_2001.to_csv('../data/clean/race_results_clean_2001-2017.csv', encoding='utf-8', index=False)

## Race Results 2018+

### Load

In [105]:
races_2018 = pd.read_csv('../data/raw/race_results_raw_2018+.csv')

### Fix circuit_id in Results

In [106]:
# Change circuit_id from 23 to 10 and from 22 to 9 
rows_replaced = (races_2018['circuit_id'] == 23).sum() + (races_2018['circuit_id'] == 22).sum()
races_2018['circuit_id'] = races_2018['circuit_id'].replace({23: 10, 22: 9})

print(f"Rows replaced: {rows_replaced}")

Rows replaced: 60


### Separate Position Status

In [107]:
races_2018['end_position'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', 'NC', '16', '17', '18', '19', '20', 'DQ'],
      dtype=object)

In [108]:
# Initialize new columns
end_positions = []
statuses = []
current_position = 1
current_race = None

# Group by each race
for idx, row in races_2018.iterrows():
    # Check if its a new race
    if current_race != row['race_url']:
        current_race = row['race_url']
        current_position = 1

    pos = row['end_position']

    try:
        # Try converting to int to validate position
        numeric_pos = int(pos)
        end_positions.append(numeric_pos)
        statuses.append('CLAS')
        current_position = numeric_pos + 1
    except ValueError:
        # Not a number so need to assign position and keep status
        if pos in ['NC', 'DQ']:
            end_positions.append(current_position)
            statuses.append(pos)
            current_position += 1
        else:
            end_positions.append(current_position)
            statuses.append('DNF')
            current_position += 1

# Assign back to dataframe
races_2018['end_position'] = end_positions
races_2018['position_status'] = statuses

print(f"End positions:\n{races_2018['end_position'].unique()}")
print(f"Position statuses:\n{races_2018['position_status'].unique()}")

End positions:
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
Position statuses:
['CLAS' 'NC' 'DQ']


### Impute NA Lap Counts

In [109]:
races_2018.isna().sum()

race_id            0
driver_id          0
circuit_id         0
team_id            0
year               0
race_url           0
circuit_name       0
driver_name        0
team_name          0
end_position       0
points             0
laps_completed     5
position_status    0
dtype: int64

In [110]:
# Impute 0 for laps_completed if null
races_2018['laps_completed'] = races_2018['laps_completed'].fillna(0)
races_2018['laps_completed'].isna().sum()

0

### Map Team Name

In [111]:
print(f"Unique team names before mapping:\n{races_2018['team_name'].unique()}")

# Map team names to constructor common names using existing constructor_mapping
races_2018['team_name'] = races_2018['team_name'].map(constructor_mapping['team_id']).fillna(races_2018['team_name'])

print(f"Unique team names after mapping:\n{races_2018['team_name'].unique()}")

Unique team names before mapping:
['Ferrari' 'Mercedes' 'Red Bull Racing TAG Heuer' 'McLaren Renault'
 'Renault' 'Force India Mercedes' 'Sauber Ferrari' 'Williams Mercedes'
 'Scuderia Toro Rosso Honda' 'Haas Ferrari' 'Red Bull Racing Honda'
 'Alfa Romeo Racing Ferrari' 'Racing Point BWT Mercedes'
 'AlphaTauri Honda' 'McLaren Mercedes' 'Aston Martin Mercedes'
 'Alpine Renault' 'Alfa Romeo Ferrari' 'AlphaTauri RBPT'
 'Aston Martin Aramco Mercedes' 'Red Bull Racing RBPT'
 'Red Bull Racing Honda RBPT' 'AlphaTauri Honda RBPT'
 'Kick Sauber Ferrari' 'RB Honda RBPT' 'McLaren' 'Red Bull Racing'
 'Williams' 'Aston Martin' 'Kick Sauber' 'Alpine' 'Racing Bulls'
 'Haas F1 Team']
Unique team names after mapping:
['Ferrari' 'Mercedes' 'Red Bull' 'McLaren' 'Renault' 'Force India'
 'Sauber' 'Williams' 'Toro Rosso' 'Haas' 'Alfa Romeo' 'Racing Point'
 'AlphaTauri' 'Aston Martin' 'Alpine' 'Racing Bulls' 'Kick Sauber']


In [112]:
races_2018.head()

,race_id,driver_id,circuit_id,team_id,year,race_url,circuit_name,driver_name,team_name,end_position,points,laps_completed,position_status
0,1,60,1,1,2018,https://www.formula1.com/en/results/2018/races/979/australia/race-result,Australia,Sebastian Vettel,Ferrari,1,25.0,58.0,CLAS
1,1,57,1,2,2018,https://www.formula1.com/en/results/2018/races/979/australia/race-result,Australia,Lewis Hamilton,Mercedes,2,18.0,58.0,CLAS
2,1,6,1,1,2018,https://www.formula1.com/en/results/2018/races/979/australia/race-result,Australia,Kimi Räikkönen,Ferrari,3,15.0,58.0,CLAS
3,1,79,1,3,2018,https://www.formula1.com/en/results/2018/races/979/australia/race-result,Australia,Daniel Ricciardo,Red Bull,4,12.0,58.0,CLAS
4,1,12,1,4,2018,https://www.formula1.com/en/results/2018/races/979/australia/race-result,Australia,Fernando Alonso,McLaren,5,10.0,58.0,CLAS


### Save

In [113]:
races_2018.to_csv('../data/clean/race_results_clean_2018+.csv', encoding='utf-8', index=False)

## Practice Results

### Load

In [94]:
practices = pd.read_csv('../data/raw/pratice_results_raw.csv')

In [95]:
practices.head(100)

,race_id,driver_id,team_id,session_type,lap_time,lap_count,position
0,1,57,2,practice1,1:24.026,27,1
1,1,83,2,practice1,+0.551s,29,2
2,1,94,3,practice1,+0.745s,26,3
3,1,6,1,practice1,+0.849s,23,4
4,1,60,1,practice1,+0.969s,22,5
5,1,79,3,practice1,+1.037s,25,6
6,1,67,10,practice1,+1.704s,24,7
7,1,12,4,practice1,+1.870s,16,8
8,1,93,5,practice1,+1.896s,23,9
9,1,100,4,practice1,+2.456s,15,10


### Add Completed Lap Time Column

In [96]:
practices['recorded_lap_time'] = practices['lap_time'].notna()
practices.head()

,race_id,driver_id,team_id,session_type,lap_time,lap_count,position,recorded_lap_time
0,1,57,2,practice1,1:24.026,27,1,True
1,1,83,2,practice1,+0.551s,29,2,True
2,1,94,3,practice1,+0.745s,26,3,True
3,1,6,1,practice1,+0.849s,23,4,True
4,1,60,1,practice1,+0.969s,22,5,True


### Convert Lap Times

In [97]:
# Initialize variables to track base time for each race_id and session_type
current_race_id = None
current_session_type = None
base_time = None

for idx, row in practices.iterrows():
    lap_time = row['lap_time']
    
    # Check if starting a new race_id and session_type group
    if current_race_id != row['race_id'] or current_session_type != row['session_type']:
        current_race_id = row['race_id']
        current_session_type = row['session_type']
        base_time = None
    
    try:
        if pd.notna(lap_time):
            # Check if this is a base time
            if not lap_time.startswith('+'):
                if ':' in lap_time:
                    # Time in "min:sec.millisec" format
                    time_parts = re.split(r"[:.]", lap_time)
                    minutes = int(time_parts[0])
                    seconds = int(time_parts[1])
                    milliseconds = int(time_parts[2])
                else:
                    # Time in "sec.millisec" format
                    time_parts = lap_time.split('.')
                    minutes = 0
                    seconds = int(time_parts[0])
                    milliseconds = int(time_parts[1])
                
                # Convert to timedelta and store as base time
                base_time = timedelta(minutes=minutes, seconds=seconds, milliseconds=milliseconds)
                practices.at[idx, 'lap_time_clean'] = base_time
            else:
                if base_time is not None:
                    # Get rid of the + and s
                    time_clean = lap_time.strip('+s')
                    
                    # Parse the gap time
                    if ':' in time_clean:
                        # Gap time in "min:sec.millisec" format
                        time_parts = re.split(r"[:.]", time_clean)
                        gap_minutes = int(time_parts[0])
                        gap_seconds = int(time_parts[1])
                        gap_milliseconds = int(time_parts[2])
                    else:
                        # Gap time in "sec.millisec" format
                        time_parts = time_clean.split('.')
                        gap_minutes = 0
                        gap_seconds = int(time_parts[0])
                        gap_milliseconds = int(time_parts[1])
                    
                    # Convert gap to timedelta and add to base time
                    gap = timedelta(minutes=gap_minutes, seconds=gap_seconds, milliseconds=gap_milliseconds)
                    new_time = base_time + gap
                    practices.at[idx, 'lap_time_clean'] = new_time
    
    except (ValueError, AttributeError):
        if pd.isna(lap_time):
            practices.at[idx, 'lap_time_clean'] = None
            continue
        else:
            # Handle unexpected format
            practices.at[idx, 'lap_time_clean'] = None

In [98]:
practices.head()

,race_id,driver_id,team_id,session_type,lap_time,lap_count,position,recorded_lap_time,lap_time_clean
0,1,57,2,practice1,1:24.026,27,1,True,0 days 00:01:24.026000
1,1,83,2,practice1,+0.551s,29,2,True,0 days 00:01:24.577000
2,1,94,3,practice1,+0.745s,26,3,True,0 days 00:01:24.771000
3,1,6,1,practice1,+0.849s,23,4,True,0 days 00:01:24.875000
4,1,60,1,practice1,+0.969s,22,5,True,0 days 00:01:24.995000


### Impute Missing Lap Times

In [99]:
practices.isna().sum()

race_id                0
driver_id              0
team_id                0
session_type           0
lap_time             155
lap_count              0
position               0
recorded_lap_time      0
lap_time_clean       155
dtype: int64

In [100]:
# Group by race_id and session_type
for (race_id, session_type), group in practices.groupby(['race_id', 'session_type']):
    if group['lap_time_clean'].isna().any():
        
        # Get the most recent non-null value in this group if any
        most_recent_time = group['lap_time_clean'].dropna().iloc[-1] if not group['lap_time_clean'].dropna().empty else None
        
        if most_recent_time is not None:
            imputed_time = most_recent_time * 1.05
            
            # Impute the time to missing values in this group
            missing_indices = group[group['lap_time_clean'].isna()].index
            practices.loc[missing_indices, 'lap_time_clean'] = imputed_time

practices.isna().sum()

race_id                0
driver_id              0
team_id                0
session_type           0
lap_time             155
lap_count              0
position               0
recorded_lap_time      0
lap_time_clean         0
dtype: int64

In [101]:
practices.drop(['lap_time', 'team_id'], axis=1, inplace=True)

### Format Session Type

In [102]:
print(f"Current session values:\n{practices['session_type'].unique()}")

session_map = {
    'practice1': 'FP1',
    'practice2': 'FP2',
    'practice3': 'FP3'
}

practices['session_type'] = practices['session_type'].map(session_map)

print(f"\nNew session values:\n{practices['session_type'].unique()}")

Current session values:
['practice1' 'practice2' 'practice3']

New session values:
['FP1' 'FP2' 'FP3']


### Convert Long to Wide

In [103]:
# Pivot the practices DataFrame to create separate columns for each session type
practices_pivot = practices.pivot_table(
    index=['race_id', 'driver_id'],
    columns='session_type',
    values=['lap_time_clean', 'lap_count', 'position', 'recorded_lap_time'],
    aggfunc='first'
)

# Flatten the multi-level column index
practices_pivot.columns = [f'{col[0]}_{col[1]}' for col in practices_pivot.columns]

# Reset index to make race_id and driver_id regular columns
practices_aggregated = practices_pivot.reset_index()

practices_aggregated.head()

,race_id,driver_id,lap_count_FP1,lap_count_FP2,lap_count_FP3,lap_time_clean_FP1,lap_time_clean_FP2,lap_time_clean_FP3,position_FP1,position_FP2,position_FP3,recorded_lap_time_FP1,recorded_lap_time_FP2,recorded_lap_time_FP3
0,1,6,23.0,39.0,13.0,0 days 00:01:24.875000,0 days 00:01:24.214000,0 days 00:01:28.499000,4.0,4.0,2.0,True,True,True
1,1,12,16.0,28.0,13.0,0 days 00:01:25.896000,0 days 00:01:25.200000,0 days 00:01:34.298000,8.0,8.0,10.0,True,True,True
2,1,57,27.0,35.0,7.0,0 days 00:01:24.026000,0 days 00:01:23.931000,0 days 00:01:34.225000,1.0,1.0,8.0,True,True,True
3,1,60,22.0,38.0,15.0,0 days 00:01:24.995000,0 days 00:01:24.451000,0 days 00:01:26.067000,5.0,5.0,1.0,True,True,True
4,1,67,24.0,34.0,6.0,0 days 00:01:25.730000,0 days 00:01:24.648000,0 days 00:01:36.171000,7.0,6.0,15.0,True,True,True


### Save

In [104]:
practices_aggregated.to_csv('../data/clean/practice_results_clean.csv', encoding='utf-8', index=False)

## Qualifying Results

### Load

In [170]:
qualifying = pd.read_csv('../data/raw/qualifying_results_raw.csv')

### Drop Extra Columns

In [171]:
qualifying.drop(['team_id', 'qual_laps'], axis=1, inplace=True)

### Convert Non-Numeric Places

In [172]:
# Initialize new columns
qual_positions = []
current_position = 1
current_race = None

# Group by each race
for idx, row in qualifying.iterrows():
    # Check if its a new race
    if current_race != row['race_id']:
        current_race = row['race_id']
        current_position = 1

    pos = row['qual_position']

    try:
        # Try converting to int to validate position
        numeric_pos = int(pos)
        qual_positions.append(numeric_pos)
        current_position = numeric_pos + 1
    except ValueError:
        # Not a number so assign next available position
        qual_positions.append(current_position)
        current_position += 1

# Assign back to dataframe
qualifying['qual_position'] = qual_positions

print(f"Qualifying positions:\n{qualifying['qual_position'].unique()}")

Qualifying positions:
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]


### No Lap Time Columns

In [ ]:
def clean_qualifying_times(df):
    """
    Cleans qualifying session times and adds no_lap_time flags for Q1, Q2, Q3.
    Uses qualifying position to determine advancement and flags drivers with DNF/DNS/NA times.
    
    Assumes df contains: ['race_id', 'driver_id', 'q1_time', 'q2_time', 'q3_time', 'qual_position']
    """
    
    df = df.copy()
    
    # Define cutoffs
    Q2_CUTOFF = 15  # Top 15 advance to Q2
    Q3_CUTOFF = 10  # Top 10 advance to Q3
    
    # Ensure qualifying position is numeric and sort
    df['qual_position'] = pd.to_numeric(df['qual_position'], errors='coerce')
    df = df.sort_values(['race_id', 'qual_position'])
    
    # Determine who advances to Q2 and Q3
    df['advanced_to_q2'] = df['qual_position'] <= Q2_CUTOFF
    df['advanced_to_q3'] = df['qual_position'] <= Q3_CUTOFF
    
    # Convert time strings to seconds
    for session in ['q1', 'q2', 'q3']:
        time_col = f'{session}_time'
        
        # Convert to numeric
        df[time_col] = df[time_col].apply(lambda x: 
            None if pd.isna(x) or (isinstance(x, str) and x.upper() in ['DNF', 'DNS', '']) 
            else float(x.split(':')[0]) * 60 + float(x.split(':')[1]) if isinstance(x, str) and ':' in x 
            else float(x) if isinstance(x, str) 
            else x
        )
    
    # Add no_lap_time flags for each session
    df['q1_no_lap_time_flag'] = df['q1_time'].isna().astype(int)
    df['q2_no_lap_time_flag'] = ((df['advanced_to_q2']) & (df['q2_time'].isna())).astype(int)
    df['q3_no_lap_time_flag'] = ((df['advanced_to_q3']) & (df['q3_time'].isna())).astype(int)
    
    # Get max times per session for imputation
    df['max_q1_time'] = df.groupby('race_id')['q1_time'].transform('max')
    df['max_q2_time'] = df.groupby('race_id')['q2_time'].transform('max')
    df['max_q3_time'] = df.groupby('race_id')['q3_time'].transform('max')
    
    # Q1 imputation
    df.loc[df['q1_time'].isna(), 'q1_time'] = df['max_q1_time'] + 5
    
    # Q2 imputation
    q2_missing = df['q2_time'].isna()
    df.loc[q2_missing & df['advanced_to_q2'], 'q2_time'] = df['max_q2_time'] + 5  # +5s for drivers who advanced but didnt set a time
    df.loc[q2_missing & ~df['advanced_to_q2'], 'q2_time'] = df['max_q2_time'] + 10  # +10s for drivers who didnt advance
    
    # Q3 imputation 
    q3_missing = df['q3_time'].isna()
    df.loc[q3_missing & df['advanced_to_q3'], 'q3_time'] = df['max_q3_time'] + 5
    df.loc[q3_missing & ~df['advanced_to_q3'], 'q3_time'] = df['max_q3_time'] + 10
    
    # Drop helper columns
    df.drop(columns=['advanced_to_q2', 'advanced_to_q3', 
                     'max_q1_time', 'max_q2_time', 'max_q3_time'], inplace=True)
    
    return df

In [174]:
qualifying_cleaned = clean_qualifying_times(qualifying)
qualifying_cleaned.head(20)

,race_id,driver_id,q1_time,q2_time,q3_time,qual_position,q1_no_lap_time_flag,q2_no_lap_time_flag,q3_no_lap_time_flag
0,1,57,82.824,82.051,81.164,1,0,0,0
1,1,6,83.096,82.507,81.828,2,0,0,0
2,1,60,83.348,81.944,81.838,3,0,0,0
3,1,94,83.483,82.416,81.879,4,0,0,0
4,1,79,83.494,82.897,82.152,5,0,0,0
5,1,87,83.909,83.300,83.187,6,0,0,0
6,1,67,83.671,83.468,83.339,7,0,0,0
7,1,70,83.782,83.544,83.532,8,0,0,0
8,1,93,83.529,83.061,83.577,9,0,0,0
9,1,83,83.686,82.089,88.577,10,0,0,1


### Correct DataTypes

In [175]:
print(f"Previous DataTypes:\n{qualifying_cleaned.dtypes}")

qualifying_cleaned['q1_no_lap_time_flag'] = qualifying_cleaned['q1_no_lap_time_flag'].astype(bool)
qualifying_cleaned['q2_no_lap_time_flag'] = qualifying_cleaned['q2_no_lap_time_flag'].astype(bool)
qualifying_cleaned['q3_no_lap_time_flag'] = qualifying_cleaned['q3_no_lap_time_flag'].astype(bool)

qualifying_cleaned['q1_time'] = pd.to_timedelta(qualifying_cleaned['q1_time'], unit='s')
qualifying_cleaned['q2_time'] = pd.to_timedelta(qualifying_cleaned['q2_time'], unit='s')
qualifying_cleaned['q3_time'] = pd.to_timedelta(qualifying_cleaned['q3_time'], unit='s')

print(f"\nNew DataTypes:\n{qualifying_cleaned.dtypes}")

display(qualifying_cleaned.head())

Previous DataTypes:
race_id                  int64
driver_id                int64
q1_time                float64
q2_time                float64
q3_time                float64
qual_position            int64
q1_no_lap_time_flag      int32
q2_no_lap_time_flag      int32
q3_no_lap_time_flag      int32
dtype: object

New DataTypes:
race_id                          int64
driver_id                        int64
q1_time                timedelta64[ns]
q2_time                timedelta64[ns]
q3_time                timedelta64[ns]
qual_position                    int64
q1_no_lap_time_flag               bool
q2_no_lap_time_flag               bool
q3_no_lap_time_flag               bool
dtype: object


,race_id,driver_id,q1_time,q2_time,q3_time,qual_position,q1_no_lap_time_flag,q2_no_lap_time_flag,q3_no_lap_time_flag
0,1,57,0 days 00:01:22.824000,0 days 00:01:22.051000,0 days 00:01:21.164000,1,False,False,False
1,1,6,0 days 00:01:23.096000,0 days 00:01:22.507000,0 days 00:01:21.828000,2,False,False,False
2,1,60,0 days 00:01:23.348000,0 days 00:01:21.944000,0 days 00:01:21.838000,3,False,False,False
3,1,94,0 days 00:01:23.483000,0 days 00:01:22.416000,0 days 00:01:21.879000,4,False,False,False
4,1,79,0 days 00:01:23.494000,0 days 00:01:22.897000,0 days 00:01:22.152000,5,False,False,False


### Save

In [176]:
qualifying_cleaned.to_csv('../data/clean/qualifying_results_clean.csv', encoding='utf-8', index=False)

## Starting Grid

### Load

In [10]:
starting = pd.read_csv('../data/raw/starting_grid_results_raw.csv')

### Drop Excess Column

In [11]:
starting.drop('team_id', axis=1, inplace=True)
starting.head()

,race_id,driver_id,start_position
0,1,57,1
1,1,6,2
2,1,60,3
3,1,94,4
4,1,87,5


### Save

In [12]:
starting.to_csv('../data/clean/starting_grid_clean.csv', encoding='utf-8', index=False)

## Pit Stops

## Fastest Laps

## Rounds

## Lap Results

## Weather

## Flag Results

## Circuits

### Load

In [23]:
circuits = pd.read_csv('../data/raw/circuits_raw.csv')

### Separate Bahrain Outer Circuit

In [24]:
# Create a dictionary with Sakhir Outer Circuit (2020) data
new_circuit_data = {
    'name': 'Bahrain International Outer Circuit',
    'type': 'Race circuit',
    'direction': 'Clockwise',
    'location': 'Sakhir',
    'country': 'Bahrain',
    'length': '3.543 km (2.202 mi)',
    'turns': '11',
    'gp': 'Sakhir Grand Prix',
    'seasons': '2020',
    'gps_held': '1'
}

# Convert to DataFrame and append to existing data
new_row = pd.DataFrame([new_circuit_data])
circuits = pd.concat([circuits, new_row], ignore_index=True)

circuits.tail(3)

,name,type,direction,location,country,length,turns,gp,seasons,gps_held
76,Yas Marina Circuit *,Race circuit,Anti-clockwise,Abu Dhabi,United Arab Emirates,5.281 km (3.281 mi),15,Abu Dhabi Grand Prix,2009–2024,16
77,Zeltweg Airfield,Road circuit,Clockwise,Zeltweg,Austria,3.186 km (1.980 mi),4,Austrian Grand Prix,1964,1
78,Bahrain International Outer Circuit,Race circuit,Clockwise,Sakhir,Bahrain,3.543 km (2.202 mi),11,Sakhir Grand Prix,2020,1


### Filter for 2018+ Seasons

In [25]:
circuits = circuits.dropna(subset=['seasons'])

def has_year_after_2018(season_str, target_start=2018):
    # Split by comma to handle multiple ranges
    ranges = season_str.split(',')
    
    for range_part in ranges:
        range_part = range_part.strip()
        
        # Clean the string
        range_part = range_part.replace(' ', '').replace('[', '').replace(']', '').replace('c', '').replace('e', '')
        
        # Check if it's a range
        if '–' in range_part:
            end_year = int(range_part.split('–')[-1])
        else:
            # Single year with no range
            end_year = int(range_part)
        
        # Check if this range includes any year from 2018 onward
        if end_year >= target_start:
            return True
    
    return False

circuits = circuits[circuits['seasons'].apply(has_year_after_2018)]
print(f"{circuits.shape[0]} rows left in the filtered dataframe")

32 rows left in the filtered dataframe


### Add circuit_id Column

In [26]:
# Load circuit_id mapping dictionary
circuit_id_map = load_id_map('../data/raw/circuit_id_map.pkl')

def find_circuit_info(gp_str, country_str, id_map):
    # Clean the GP string
    cleaned_gp = gp_str.replace("Grand Prix", "").replace("\n", "").strip()
    
    # Split on commas if present
    gp_parts = [part.strip() for part in cleaned_gp.split(',')] if ',' in cleaned_gp else [cleaned_gp]
    
    # Try to match each part with circuit_id dictionary keys
    for part in gp_parts:
        for key in id_map.keys():
            if part.lower() in key.lower() or key.lower() in part.lower():
                return id_map[key], key
    
    # If no match found in GP parts, try to match with country
    for key in id_map.keys():
        if country_str.lower() in key.lower() or key.lower() in country_str.lower():
            return id_map[key], key
    
    # If still no match found
    return "no match", "no match"

# Apply the function to create circuit_id and circuit_name columns
circuits[['circuit_id', 'circuit_name']] = circuits.apply(
    lambda row: pd.Series(find_circuit_info(row['gp'], row['country'], circuit_id_map)), 
    axis=1
)

circuits

,name,type,direction,location,country,length,turns,gp,seasons,gps_held,circuit_id,circuit_name
3,Albert Park Circuit *,Street circuit,Clockwise,Melbourne,Australia,5.278 km (3.280 mi),14,Australian Grand Prix,"1996–2019, 2022–2025",28,1,Australia
4,Algarve International Circuit,Race circuit,Clockwise,Portimão,Portugal,4.653 km (2.891 mi),15,Portuguese Grand Prix,2020–2021,2,26,Portugal
6,Autódromo Hermanos Rodríguez *,Race circuit,Clockwise,Mexico City,Mexico,4.304 km (2.674 mi),17,"Mexican Grand Prix,\nMexico City Grand Prix","1963–1970, 1986–1992, 2015–2019, 2021–2024",24,19,Mexico
8,Autodromo Internazionale del Mugello,Race circuit,Clockwise,Scarperia e San Piero,Italy,5.245 km (3.259 mi),15,Tuscan Grand Prix,2020,1,24,Tuscany
9,Autodromo Internazionale Enzo e Dino Ferrari *,Race circuit,Anti-clockwise,Imola,Italy,4.909 km (3.050 mi),19,"Italian Grand Prix,\nSan Marino Grand Prix,\nE...","1980–2006, 2020–2022, 2024–2025",32,14,Italy
10,Autodromo José Carlos Pace *,Race circuit,Anti-clockwise,São Paulo,Brazil,4.309 km (2.677 mi),15,"Brazilian Grand Prix,\nSão Paulo Grand Prix","1973–1977, 1979–1980, 1990–2019, 2021–2024",41,20,Brazil
11,Autodromo Nazionale di Monza *,Race circuit,Clockwise,Monza,Italy,5.793 km (3.600 mi),11,Italian Grand Prix,"1950–1979, 1981–2025",75,14,Italy
14,Bahrain International Circuit *,Race circuit,Clockwise,Sakhir,Bahrain,5.412 km (3.363 mi),15,"Bahrain Grand Prix,\nSakhir Grand Prix","2004–2010, 2012–2025[c]",22,2,Bahrain
15,Baku City Circuit *,Street circuit,Anti-clockwise,Baku,Azerbaijan,6.003 km (3.730 mi),20,"European Grand Prix,\nAzerbaijan Grand Prix","2016–2019, 2021–2025",9,4,Azerbaijan
22,Circuit de Barcelona-Catalunya *,Race circuit,Clockwise,Montmeló,Spain,4.657 km (2.894 mi),14,Spanish Grand Prix,1991–2025,35,5,Spain


In [27]:
# Update circuit names and IDs for specific rows
circuit_id_map = load_id_map('../data/raw/circuit_id_map.pkl')

# Update row index 9: change circuit name to "Emilia-Romagna"
new_circuit_id_9, new_circuit_name_9 = find_circuit_info("Emilia-Romagna", circuits.loc[9, 'country'], circuit_id_map)
circuits.loc[9, 'circuit_name'] = new_circuit_name_9
circuits.loc[9, 'circuit_id'] = new_circuit_id_9

# Update row index 59: change circuit name to "Eifel"
new_circuit_id_59, new_circuit_name_59 = find_circuit_info("Eifel", circuits.loc[59, 'country'], circuit_id_map)
circuits.loc[59, 'circuit_name'] = new_circuit_name_59
circuits.loc[59, 'circuit_id'] = new_circuit_id_59

# Check for duplicates in circuit_id and circuit_name columns
duplicate_circuit_ids = circuits[circuits.duplicated('circuit_id', keep=False)]
duplicate_circuit_names = circuits[circuits.duplicated('circuit_name', keep=False)]

print(f"Rows with duplicate circuit_id: {duplicate_circuit_ids.shape[0]}")
print(f"Rows with duplicate circuit_name: {duplicate_circuit_names.shape[0]}")

print(f"\nUpdated row 9: circuit_name = '{new_circuit_name_9}', circuit_id = {new_circuit_id_9}")
print(f"Updated row 59: circuit_name = '{new_circuit_name_59}', circuit_id = {new_circuit_id_59}")

Rows with duplicate circuit_id: 0
Rows with duplicate circuit_name: 0

Updated row 9: circuit_name = 'Emilia-Romagna', circuit_id = 27
Updated row 59: circuit_name = 'Eifel', circuit_id = 25


### Remove Unnecessary Data

In [28]:
circuits = circuits[['name', 'type', 'direction', 'length', 'turns', 'circuit_id', 'circuit_name']].sort_values('circuit_id', ascending=False)
circuits['length'] = circuits['length'].apply(lambda x: x.split(' ')[0] if isinstance(x, str) else x)
circuits

,name,type,direction,length,turns,circuit_id,circuit_name
50,Las Vegas Strip Circuit *,Street circuit,Anti-clockwise,6.201,17,34,Las Vegas
55,Miami International Autodrome *,Street circuit,Anti-clockwise,5.412,19,33,Miami
47,Jeddah Corniche Circuit *,Street circuit,Anti-clockwise,6.174,27,32,Saudi Arabia
52,Lusail International Circuit *,Race circuit,Clockwise,5.419,16,31,Qatar
33,Circuit Zandvoort *,Race circuit,Clockwise,4.259,14,30,Netherlands
78,Bahrain International Outer Circuit,Race circuit,Clockwise,3.543,11,29,Sakhir
46,Intercity Istanbul Park,Race circuit,Anti-clockwise,5.338,14,28,Turkey
9,Autodromo Internazionale Enzo e Dino Ferrari *,Race circuit,Anti-clockwise,4.909,19,27,Emilia-Romagna
4,Algarve International Circuit,Race circuit,Clockwise,4.653,15,26,Portugal
59,Nürburgring,Race circuit,Clockwise,5.148,15,25,Eifel


### Fix Japan Direction

In [29]:
circuits.loc[circuits['circuit_name'] == 'Japan', 'direction'] = 'Figure eight'
circuits[circuits['circuit_name'] == 'Japan']

,name,type,direction,length,turns,circuit_id,circuit_name
72,Suzuka International Racing Course *,Race circuit,Figure eight,5.807,18,17,Japan


### Clean Name

In [30]:
circuits['name'] = circuits['name'].str.replace('*', '').str.strip()
circuits.loc[63]

name            Red Bull Ring
type             Race circuit
direction           Clockwise
length                  4.318
turns                      10
circuit_id                  9
circuit_name          Austria
Name: 63, dtype: object

### Correct DataTypes

In [31]:
print(f"Previous DataTypes:\n{circuits.dtypes}")

circuits['length'] = circuits['length'].astype(float)
circuits['turns'] = circuits['turns'].astype(int)
circuits['circuit_id'] = circuits['circuit_id'].astype(int)


print(f"\nNew DataTypes:\n{circuits.dtypes}")

Previous DataTypes:
name            object
type            object
direction       object
length          object
turns           object
circuit_id       int64
circuit_name    object
dtype: object

New DataTypes:
name             object
type             object
direction        object
length          float64
turns             int32
circuit_id        int32
circuit_name     object
dtype: object


### Save

In [32]:
circuits.to_csv('../data/clean/circuits_clean.csv', encoding='utf-8', index=False)

## Location

### Load

In [36]:
locations = pd.read_csv('../data/raw/locations_raw.csv')

### Remove Unnecessary Data

In [37]:
locations = locations[['cleaned_name', 'elevation']]
locations = locations.dropna(subset=['elevation'])
locations = locations.rename(columns={'cleaned_name': 'name'})

### Correct Datatypes

In [ ]:
print(f"Previous DataTypes:\n{locations.dtypes}")

locations['elevation'] = locations['elevation'].astype(float)

print(f"\nNew DataTypes:\n{locations.dtypes}")

Previous DataTypes:
name          object
elevation    float64
dtype: object

New DataTypes:
name          object
elevation    float64
dtype: object


### Save

In [39]:
locations.to_csv('../data/clean/locations_clean.csv', encoding='utf-8', index=False)

# Post-Merge Cleaning

### Calculate Rolling Average Finish

In [59]:
# Define rolling windows
windows = [3, 5, 10]

# Sort so windows are applied chronologically
races_2001 = races_2001.sort_values(['driver_id', 'date'])

# Create rolling average finish columns
for w in windows:
    races_2001[f'avg_finish_last_{w}'] = (
        races_2001.groupby('driver_id')['position']
        .transform(lambda x: x.shift(1).rolling(w, min_periods=1).mean())
    )
races_2001.head()

,date,driver_id,position,driver_name,points,avg_finish_last_3,avg_finish_last_5,avg_finish_last_10
0,2001-03-04,1,1,Michael Schumacher,10.0,NaN,NaN,NaN
22,2001-03-18,1,1,Michael Schumacher,10.0,1.000000,1.000000,1.000000
45,2001-04-01,1,2,Michael Schumacher,6.0,1.000000,1.000000,1.000000
84,2001-04-15,1,19,Michael Schumacher,0.0,1.333333,1.333333,1.333333
88,2001-04-29,1,1,Michael Schumacher,10.0,7.333333,5.750000,5.750000


### Aggregate Stats

In [60]:
# Aggregate base driver stats
stats_dict = {
    'total_wins': ('position', lambda x: (x == 1).sum()),
    'total_podiums': ('position', lambda x: x.isin([1, 2, 3]).sum()),
    'total_points': ('points', 'sum')
}

# Add rolling average finish columns
for w in windows:
    stats_dict[f'avg_finish_last_{w}'] = (f'avg_finish_last_{w}', 'last')

# Run groupby aggregation
driver_stats = races_2001.groupby(['driver_id', 'driver_name']).agg(**stats_dict).reset_index()

driver_stats.head()

,driver_id,driver_name,total_wins,total_podiums,total_points,avg_finish_last_3,avg_finish_last_5,avg_finish_last_10
0,1,Michael Schumacher,47,72,888.0,16.333333,14.6,13.8
1,2,David Coulthard,4,21,241.0,12.333333,12.8,13.4
2,3,Rubens Barrichello,10,53,519.0,13.000000,13.8,14.6
3,4,Nick Heidfeld,0,13,259.0,14.000000,14.0,11.2
4,5,Heinz-Harald Frentzen,0,1,21.0,10.000000,11.8,13.9
